### 1. Написать приложение, которое собирает основные новости с сайта на выбор news.mail.ru, lenta.ru, yandex-новости. Для парсинга использовать XPath. Структура данных должна содержать:
- название источника;
- наименование новости;
- ссылку на новость;
- дата публикации.
### 2. Сложить собранные новости в БД
- Минимум один сайт, максимум - все три

In [78]:
import requests
from pprint import pprint
from lxml import html
from datetime import date
from pymongo import MongoClient
import pandas as pd

In [79]:
client = MongoClient('localhost', 27017)
db = client['parcing_news']
collection = db['news']

In [80]:
news_site = 'https://news.mail.ru/'
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36'}
response = requests.get(news_site, headers=header)
dom = html.fromstring(response.text)
elements = dom.xpath('//div[contains(@class, "newsitem " )]')



for i in elements:
    position = {}
    name = i.xpath('.//a/span/text()')[0].replace("\xa0", " ")
    link = i.xpath('.//a/@href')[0]
    time = f"{date.today()} {i.xpath('./div/span/text()')[0]}"
    source = i.xpath('./div/span/text()')[1]

    position["Новость"] = name
    position["Ссылка"] = link
    position["Время публикации"] = time
    position["Источник"] = source
    position["Сайт новостей"] = news_site

    if not collection.find_one({'Ссылка': position['Ссылка']}):
        collection.insert_one(position)


In [81]:
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36'}
news_site = 'https://lenta.ru'
response = requests.get(news_site, headers=header)
dom = html.fromstring(response.text)

elements = dom.xpath('//div[@class="item"]')


for i in elements:
    position = {}
    name = i.xpath('./a/text()')[0]
    link = f"{news_site}{i.xpath('.//a/@href')[0]}"
    try:
        time = i.xpath('./a/time/@datetime')[0]
    except:
        time = None


    position["Новость"] = name
    position["Ссылка"] = link
    position["Время публикации"] = time
    position["Источник"] = None
    position["Сайт новостей"] = news_site

    if not collection.find_one({'Ссылка': position['Ссылка']}):
        collection.insert_one(position)




In [82]:
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36'}
news_site = 'https://yandex.ru/news'
response = requests.get(news_site, headers=header)
dom = html.fromstring(response.text)
elements = dom.xpath('//article')

for i in elements:
    position = {}
    name = i.xpath('.//div[@class="mg-card__annotation"]/text()')[0]
    link = i.xpath('.//a[@class ="mg-card__link"]/@href')[0]
    sourse = i.xpath('.//a[@class="mg-card__source-link"]/text()')[0]
    time = i.xpath('.//span[@class="mg-card-source__time"]/text()')[0]
    time = f"{date.today()} {time}"


    position["Новость"] = name
    position["Ссылка"] = link
    position["Время публикации"] = time
    position["Источник"] = sourse
    position["Сайт новостей"] = news_site

    if not collection.find_one({'Ссылка': position['Ссылка']}):
        collection.insert_one(position)

In [83]:
df = pd.DataFrame([i for i in collection.find({})])

In [84]:
df

,_id,Новость,Ссылка,Время публикации,Источник,Сайт новостей
0,61816d411cae1ae47490a2e2,МЧС: Туман продержится в Москве до конца вторника,https://pogoda.mail.ru/news/48613371/,2021-11-02 17:58 (мск),Агентство городских новостей «Москва»,https://news.mail.ru/
1,61816d411cae1ae47490a2e3,Восемь линий столичного метро планируют продли...,https://news.mail.ru/society/48612334/,2021-11-02 17:32 (мск),m24.ru,https://news.mail.ru/
2,61816d411cae1ae47490a2e4,"Путин заявил, что российские военные научились...",https://news.mail.ru/politics/48610008/,2021-11-02 16:18 (мск),ТАСС,https://news.mail.ru/
3,61816d411cae1ae47490a2e5,Создатели криптовалюты по «Игре в кальмара» ок...,https://news.mail.ru/economics/48607854/,2021-11-02 13:59 (мск),Газета.Ру,https://news.mail.ru/
4,61816d411cae1ae47490a2e6,Туман в Москве привел к пробке из самолетов в ...,https://news.mail.ru/society/48611339/,2021-11-02 16:24 (мск),РБК,https://news.mail.ru/
...,...,...,...,...,...,...
261,618175a51cae1ae47490a3f0,"Прощание с диктором, тележурналистом Игорем Ки...",https://yandex.ru/news/story/Proshhanie_sdikto...,2021-11-02 20:05,РИА Новости,https://yandex.ru/news
262,618175a91cae1ae47490a3f2,"Три человека погибли, один пострадал на федера...",https://yandex.ru/news/story/VDTP_vMurmanskoj_...,2021-11-02 20:12,ТАСС,https://yandex.ru/news
263,618175a91cae1ae47490a3f3,В целом в Новой Москве (Новомосковский и Троиц...,https://yandex.ru/news/story/Cena_novostroek_v...,2021-11-02 17:33,РБК,https://yandex.ru/news
264,618175a91cae1ae47490a3f4,Пилотный проект стартует в городе в 2022 году.,https://yandex.ru/news/story/Proekt_stroitelst...,2021-11-02 17:35,Вести.Севастополь,https://yandex.ru/news
